In [43]:
from matplotlib import pyplot as pyplot
from skimage.feature import hog
from skimage import exposure
from skimage.transform import resize

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import os
import numpy as np
import nibabel as nib



In [32]:
# colored
def process_img(filepath):
    scan = nib.load(filepath)
    scan = scan.get_fdata()
    return scan


In [47]:
# gray scale 
def process_img(filepath):
    scan = nib.load(filepath)
    scan = scan.get_fdata()
    gray_scan= np.dot(scan[..., :3], [0.2989, 0.5870, 0.1140])
    return gray_scan

In [48]:
resize_l = 250
resize_w = 250


data_folder = f'{os.getcwd()}/labeled_data/data/'

length = 0
for label in os.listdir(data_folder):
    length += len(os.listdir(f'{data_folder}/{label}/'))


# X = np.empty(shape=(length, resize_l, resize_w, 64,), dtype=np.float16)
X = []
Y = np.empty(shape=(length), dtype=np.int64)



j = 0
print('Loading Images')
for label in os.listdir(data_folder):
    img_directory = f'{data_folder}/{label}/'
    for img in os.listdir(img_directory): 

        img_path = '{}{}'.format(img_directory, img)
        img = process_img(img_path)
        img = resize(img, output_shape=(resize_l, resize_w, 64))
        img = hog(img, orientations=8, pixels_per_cell=(20,20), cells_per_block=(1,1), channel_axis=-1)
        X += [img]
        Y[j] = label
        j += 1
        

Loading Images


In [40]:
# original implementation 
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.3)


classifier = SVC()

classifier.fit(train_x, train_y)
y_pred = classifier.predict(test_x)

acc = np.mean(y_pred == test_y)
acc


0.7218543046357616

In [49]:
# k fold implmentation
from sklearn.model_selection import StratifiedKFold
n_splits = 10

X = np.array(X)
Y = np.array(Y)

skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

for fold, (train_index, test_index) in enumerate(skf.split(X, Y)):
    print(f"Fold {fold+1} ...")

    train_index = np.array(train_index).astype(int)
    test_index = np.array(test_index).astype(int)

    clf = SVC()
    clf.fit(X[train_index],Y[train_index])

    y_pred = clf.predict(X[test_index])

    acc = np.mean(y_pred == Y[test_index])
    print(acc)


Fold 1 ...
0.7128712871287128
Fold 2 ...
0.693069306930693
Fold 3 ...
0.7029702970297029
Fold 4 ...
0.7029702970297029
Fold 5 ...
0.7029702970297029
Fold 6 ...
0.71
Fold 7 ...
0.71
Fold 8 ...
0.7
Fold 9 ...
0.7
Fold 10 ...
0.7
